In [2]:
import pandas as pd
import datetime
import json

In [3]:
df = pd.read_csv('/home/juanisil/Universidad/4-Visualizacion de datos/Parcial/completo.csv', delimiter=";", encoding='utf-8')

/tmp/ipykernel_16792/1521723224.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/juanisil/Universidad/4-Visualizacion de datos/Parcial/completo.csv', delimiter=";", encoding='utf-8')


In [85]:
dataset = [[m//3 + 1, 0] for m in range(12*3)]

for i in range(len(dataset)):
    if i % 3 == 0:
        dataset[i].append("App")
    elif i % 3 == 1:
        dataset[i].append("Web")   
    else:
        dataset[i].append("Otros")



for row in df.values:
    if type(row[4]) == str:
        fecha = datetime.datetime.strptime(row[4], "%d/%m/%Y")
        if row[12] == "App BA 147" or row[12] == "App Denuncia Vial":
            dataset[(fecha.month-1)*3][1] += 1
        elif row[12] == "GCS Web":
            dataset[(fecha.month-1)*3+1][1] += 1
        else:
            dataset[(fecha.month-1)*3+2][1] += 1

In [7]:
# dataset = [Barrio, Denuncias Julio, Denuncias Agosto]
barrios = df["domicilio_barrio"].unique().tolist()[1:]
dataset = pd.DataFrame()
dataset["Barrio"] = barrios
dataset["Reportes"] = [0 for _ in range(len(barrios))]

for row in df.values:
    if not pd.isnull(row[7]):
        dataset.loc[dataset['Barrio'] == row[7], "Reportes"] += 1

In [8]:
dataset

,Barrio,Reportes
0,VILLA LURO,8292
1,RECOLETA,17655
2,VILLA GRAL. MITRE,8195
3,FLORES,31579
4,VILLA DEL PARQUE,16921
5,COGHLAN,5985
6,MATADEROS,13783
7,VELEZ SARSFIELD,7902
8,MONTE CASTRO,11066
9,PARQUE CHAS,5921


In [172]:
dataset.to_csv("data/denuncias_por_barrio.csv")

In [3]:
categorias = [
    {'name': 'TRÁNSITO', 'children' : []},
    {'name': 'LIMPIEZA Y RECOLECCIÓN', 'children' : []},
    {'name': 'OTRO', 'size' : 0},
]

for row in df.values:
    if not pd.isnull(row[0]) and not pd.isnull(row[1]):

        incategorias = False
        insubcategorias = False

        for d in categorias:
            if d['name'] == row[0]:
                incategorias = True
                for sub in d['children']:
                    if sub['name'] == row[1]:
                        insubcategorias = True
                        sub['size'] += 1
                if not insubcategorias:
                    d['children'].append({'name': row[1], 'size': 1})

        if not incategorias:
            categorias[2]['size'] += 1


In [4]:
flare = json.dumps({'name': 'Categorias', 'children': categorias}, ensure_ascii=False)

In [5]:
with open('categorias.json', 'w') as outfile:
    outfile.write(flare)